<a href="https://colab.research.google.com/github/Tednn493/football_match_prediction/blob/main/football_Machine_learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [111]:
import pandas as pd

In [112]:
matches = pd.read_csv("matches.csv", index_col=0)

In [113]:
matches.head()

,date,time,comp,round,day,venue,result,gf,ga,opponent,...,match report,notes,sh,sot,dist,fk,pk,pkatt,season,team
1,2022-08-07,16:30,Premier League,Matchweek 1,Sun,Away,W,2,0,West Ham,...,Match Report,NaN,13.0,1.0,18.7,1.0,1,1,2023,Manchester City
2,2022-08-13,15:00,Premier League,Matchweek 2,Sat,Home,W,4,0,Bournemouth,...,Match Report,NaN,19.0,7.0,17.5,0.0,0,0,2023,Manchester City
3,2022-08-21,16:30,Premier League,Matchweek 3,Sun,Away,D,3,3,Newcastle Utd,...,Match Report,NaN,21.0,10.0,16.2,1.0,0,0,2023,Manchester City
4,2022-08-27,15:00,Premier League,Matchweek 4,Sat,Home,W,4,2,Crystal Palace,...,Match Report,NaN,18.0,5.0,14.1,0.0,0,0,2023,Manchester City
5,2022-08-31,19:30,Premier League,Matchweek 5,Wed,Home,W,6,0,Nott'ham Forest,...,Match Report,NaN,17.0,9.0,14.8,0.0,0,0,2023,Manchester City


In [114]:
matches.dtypes

date             object
time             object
comp             object
round            object
day              object
venue            object
result           object
gf                int64
ga                int64
opponent         object
xg              float64
xga             float64
poss            float64
attendance      float64
captain          object
formation        object
referee          object
match report     object
notes           float64
sh              float64
sot             float64
dist            float64
fk              float64
pk                int64
pkatt             int64
season            int64
team             object
dtype: object

In [115]:
matches["date"] = pd.to_datetime(matches["date"])

In [116]:
matches["venue_code"] = matches["venue"].astype("category").cat.codes

In [117]:
matches["opp_code"] = matches["opponent"].astype("category").cat.codes

In [118]:
matches["hour"] = matches["time"].str.replace(":.+", "", regex=True).astype("int")

In [119]:
matches["day_code"]=matches["date"].dt.dayofweek

In [120]:
matches["target"]=matches["result"].astype("category").cat.codes

In [121]:
matches

,date,time,comp,round,day,venue,result,gf,ga,opponent,...,fk,pk,pkatt,season,team,venue_code,opp_code,hour,day_code,target
1,2022-08-07,16:30,Premier League,Matchweek 1,Sun,Away,W,2,0,West Ham,...,1.0,1,1,2023,Manchester City,0,23,16,6,2
2,2022-08-13,15:00,Premier League,Matchweek 2,Sat,Home,W,4,0,Bournemouth,...,0.0,0,0,2023,Manchester City,1,2,15,5,2
3,2022-08-21,16:30,Premier League,Matchweek 3,Sun,Away,D,3,3,Newcastle Utd,...,1.0,0,0,2023,Manchester City,0,15,16,6,0
4,2022-08-27,15:00,Premier League,Matchweek 4,Sat,Home,W,4,2,Crystal Palace,...,0.0,0,0,2023,Manchester City,1,7,15,5,2
5,2022-08-31,19:30,Premier League,Matchweek 5,Wed,Home,W,6,0,Nott'ham Forest,...,0.0,0,0,2023,Manchester City,1,17,19,2,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38,2021-05-02,19:15,Premier League,Matchweek 34,Sun,Away,L,0,4,Tottenham,...,0.0,0,0,2021,Sheffield United,0,20,19,6,1
39,2021-05-08,15:00,Premier League,Matchweek 35,Sat,Home,L,0,2,Crystal Palace,...,1.0,0,0,2021,Sheffield United,1,7,15,5,1
40,2021-05-16,19:00,Premier League,Matchweek 36,Sun,Away,W,1,0,Everton,...,0.0,0,0,2021,Sheffield United,0,8,19,6,2
41,2021-05-19,18:00,Premier League,Matchweek 37,Wed,Away,L,0,1,Newcastle Utd,...,1.0,0,0,2021,Sheffield United,0,15,18,2,1


In [122]:
from sklearn.ensemble import RandomForestClassifier

In [199]:
rf = RandomForestClassifier(n_estimators=100, min_samples_split=30, random_state=1)

In [200]:
trainingSet= matches[matches["date"] < "2021-07-01"]
testingSet= matches[matches["date"] > "2021-07-01"]

In [201]:
predictors = ["venue_code","opp_code","hour","day_code"]

In [202]:
rf.fit(trainingSet[predictors], trainingSet["target"])

RandomForestClassifier(min_samples_split=30, n_estimators=115, random_state=1)

In [203]:
preds = rf.predict(testingSet[predictors])

In [204]:
from sklearn.metrics import accuracy_score

In [205]:
acc = accuracy_score(testingSet["target"],preds)

In [206]:
acc

0.4440789473684211

In [207]:
combined = pd.DataFrame(dict(actual=testingSet["target"],prediction=preds))

In [208]:
pd.crosstab(index=combined["actual"],columns=combined["prediction"])

prediction,0,1,2
actual,,,
0,1,162,187
1,5,328,252
2,5,234,346


In [209]:
grouped_matches=matches.groupby("team")

In [210]:
def rolling_averages(group, cols, new_cols):
  group = group.sort_values("date")
  rolling_stats=group[cols].rolling(3, closed= 'left').mean()
  group[new_cols] = rolling_stats
  group = group.dropna(subset=new_cols)
  return group

In [211]:
cols = ["gf","ga","xg","xga","poss","sh","sot","fk","pk","pkatt"]
new_cols=[f'{col}_rolling' for col in cols]

In [212]:
team1=grouped_matches.get_group("Arsenal")
team1

,date,time,comp,round,day,venue,result,gf,ga,opponent,...,fk,pk,pkatt,season,team,venue_code,opp_code,hour,day_code,target
0,2022-08-05,20:00,Premier League,Matchweek 1,Fri,Away,W,2,0,Crystal Palace,...,1.0,0,0,2023,Arsenal,0,7,20,4,2
1,2022-08-13,15:00,Premier League,Matchweek 2,Sat,Home,W,4,2,Leicester City,...,0.0,0,0,2023,Arsenal,1,11,15,5,2
2,2022-08-20,17:30,Premier League,Matchweek 3,Sat,Away,W,3,0,Bournemouth,...,0.0,0,0,2023,Arsenal,0,2,17,5,2
3,2022-08-27,17:30,Premier League,Matchweek 4,Sat,Home,W,2,1,Fulham,...,1.0,0,0,2023,Arsenal,1,9,17,5,2
4,2022-08-31,19:30,Premier League,Matchweek 5,Wed,Home,W,2,1,Aston Villa,...,1.0,0,0,2023,Arsenal,1,1,19,2,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52,2021-05-02,14:00,Premier League,Matchweek 34,Sun,Away,W,2,0,Newcastle Utd,...,1.0,0,0,2021,Arsenal,0,15,14,6,2
54,2021-05-09,19:00,Premier League,Matchweek 35,Sun,Home,W,3,1,West Brom,...,1.0,0,0,2021,Arsenal,1,22,19,6,2
55,2021-05-12,20:15,Premier League,Matchweek 36,Wed,Away,W,1,0,Chelsea,...,0.0,0,0,2021,Arsenal,0,6,20,2,2
56,2021-05-19,19:00,Premier League,Matchweek 37,Wed,Away,W,3,1,Crystal Palace,...,0.0,0,0,2021,Arsenal,0,7,19,2,2


In [213]:
matches_rolling = matches.groupby("team").apply(lambda x: rolling_averages(x, cols, new_cols))

In [214]:
matches_rolling=matches_rolling.droplevel('team')
matches_rolling

,date,time,comp,round,day,venue,result,gf,ga,opponent,...,gf_rolling,ga_rolling,xg_rolling,xga_rolling,poss_rolling,sh_rolling,sot_rolling,fk_rolling,pk_rolling,pkatt_rolling
6,2020-10-04,14:00,Premier League,Matchweek 4,Sun,Home,W,2,1,Sheffield Utd,...,2.000000,1.333333,1.433333,1.600000,50.000000,8.000000,3.666667,0.666667,0.000000,0.000000
7,2020-10-17,17:30,Premier League,Matchweek 5,Sat,Away,L,0,1,Manchester City,...,1.666667,1.666667,1.000000,1.633333,53.333333,5.666667,3.666667,0.000000,0.000000,0.000000
9,2020-10-25,19:15,Premier League,Matchweek 6,Sun,Home,L,0,1,Leicester City,...,1.000000,1.666667,0.933333,1.400000,46.666667,7.000000,3.666667,0.666667,0.000000,0.000000
11,2020-11-01,16:30,Premier League,Matchweek 7,Sun,Away,W,1,0,Manchester Utd,...,0.666667,1.000000,0.733333,0.733333,54.000000,9.666667,4.000000,1.000000,0.000000,0.000000
13,2020-11-08,19:15,Premier League,Matchweek 8,Sun,Home,L,0,3,Aston Villa,...,0.333333,0.666667,0.866667,0.800000,48.000000,9.666667,2.666667,1.000000,0.333333,0.333333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39,2023-04-29,15:00,Premier League,Matchweek 34,Sat,Away,L,0,6,Brighton,...,1.666667,0.666667,1.033333,1.433333,49.666667,11.666667,4.666667,0.666667,0.333333,0.333333
40,2023-05-06,15:00,Premier League,Matchweek 35,Sat,Home,W,1,0,Aston Villa,...,1.000000,2.666667,1.033333,2.166667,47.333333,11.333333,2.333333,0.666667,0.333333,0.333333
41,2023-05-13,15:00,Premier League,Matchweek 36,Sat,Away,L,0,2,Manchester Utd,...,1.000000,2.000000,1.000000,1.866667,40.666667,8.000000,2.000000,0.000000,0.333333,0.333333
42,2023-05-20,15:00,Premier League,Matchweek 37,Sat,Home,D,1,1,Everton,...,0.333333,2.666667,0.633333,2.666667,44.333333,7.000000,1.333333,0.000000,0.000000,0.000000


In [215]:
matches_rolling.index = range(matches_rolling.shape[0])
matches_rolling

,date,time,comp,round,day,venue,result,gf,ga,opponent,...,gf_rolling,ga_rolling,xg_rolling,xga_rolling,poss_rolling,sh_rolling,sot_rolling,fk_rolling,pk_rolling,pkatt_rolling
0,2020-10-04,14:00,Premier League,Matchweek 4,Sun,Home,W,2,1,Sheffield Utd,...,2.000000,1.333333,1.433333,1.600000,50.000000,8.000000,3.666667,0.666667,0.000000,0.000000
1,2020-10-17,17:30,Premier League,Matchweek 5,Sat,Away,L,0,1,Manchester City,...,1.666667,1.666667,1.000000,1.633333,53.333333,5.666667,3.666667,0.000000,0.000000,0.000000
2,2020-10-25,19:15,Premier League,Matchweek 6,Sun,Home,L,0,1,Leicester City,...,1.000000,1.666667,0.933333,1.400000,46.666667,7.000000,3.666667,0.666667,0.000000,0.000000
3,2020-11-01,16:30,Premier League,Matchweek 7,Sun,Away,W,1,0,Manchester Utd,...,0.666667,1.000000,0.733333,0.733333,54.000000,9.666667,4.000000,1.000000,0.000000,0.000000
4,2020-11-08,19:15,Premier League,Matchweek 8,Sun,Home,L,0,3,Aston Villa,...,0.333333,0.666667,0.866667,0.800000,48.000000,9.666667,2.666667,1.000000,0.333333,0.333333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2200,2023-04-29,15:00,Premier League,Matchweek 34,Sat,Away,L,0,6,Brighton,...,1.666667,0.666667,1.033333,1.433333,49.666667,11.666667,4.666667,0.666667,0.333333,0.333333
2201,2023-05-06,15:00,Premier League,Matchweek 35,Sat,Home,W,1,0,Aston Villa,...,1.000000,2.666667,1.033333,2.166667,47.333333,11.333333,2.333333,0.666667,0.333333,0.333333
2202,2023-05-13,15:00,Premier League,Matchweek 36,Sat,Away,L,0,2,Manchester Utd,...,1.000000,2.000000,1.000000,1.866667,40.666667,8.000000,2.000000,0.000000,0.333333,0.333333
2203,2023-05-20,15:00,Premier League,Matchweek 37,Sat,Home,D,1,1,Everton,...,0.333333,2.666667,0.633333,2.666667,44.333333,7.000000,1.333333,0.000000,0.000000,0.000000


In [216]:
def make_predictions(data, predictors):
  trainData = data[data["date"] < "2021-07-01"]
  testData = data[data["date"] > "2021-07-01"]
  rf.fit(trainData[predictors], trainData["target"])
  preds = rf.predict(testData[predictors])
  combined = pd.DataFrame(dict(actual=testData["target"],prediction=preds))
  acc = accuracy_score(testData["target"],preds)
  return acc, combined


acc, combined = make_predictions(matches_rolling, predictors + new_cols)
acc

0.48039867109634554

In [217]:
pd.crosstab(index=combined["actual"],columns=combined["prediction"])
#0 is draw, 1 is a loss, 2 is a win
#program predicts draws at an acc of 0.254
#program

prediction,0,1,2
actual,,,
0,8,186,153
1,4,385,188
2,11,240,330


In [218]:
combined = combined.merge(matches_rolling[["date","team","opponent","result"]], left_index=True,right_index=True)
combined

,actual,prediction,date,team,opponent,result
35,1,2,2021-08-13,Arsenal,Brentford,L
36,1,2,2021-08-22,Arsenal,Chelsea,L
37,1,1,2021-08-28,Arsenal,Manchester City,L
38,2,1,2021-09-11,Arsenal,Norwich City,W
39,2,1,2021-09-18,Arsenal,Burnley,W
...,...,...,...,...,...,...
2200,1,1,2023-04-29,Wolverhampton Wanderers,Brighton,L
2201,2,1,2023-05-06,Wolverhampton Wanderers,Aston Villa,W
2202,1,1,2023-05-13,Wolverhampton Wanderers,Manchester Utd,L
2203,0,1,2023-05-20,Wolverhampton Wanderers,Everton,D


In [219]:
class MissingDict(dict):
  __missing__ = lambda self, key: key


map_values= {
    "Brighton and Hove Albion": "Brighton",
    "Manchester United": "Manchester Utd",
    "Newcastle United": "Newcastle Utd",
    "West Ham United": "West Ham",
    "Wolverhampton Wanderers": "Wolves",
    "Tottenham Hotspur": "Tottenham"
}

mapping = MissingDict(**map_values)
combined["new_team"]=combined["team"].map(mapping)
combined

,actual,prediction,date,team,opponent,result,new_team
35,1,2,2021-08-13,Arsenal,Brentford,L,Arsenal
36,1,2,2021-08-22,Arsenal,Chelsea,L,Arsenal
37,1,1,2021-08-28,Arsenal,Manchester City,L,Arsenal
38,2,1,2021-09-11,Arsenal,Norwich City,W,Arsenal
39,2,1,2021-09-18,Arsenal,Burnley,W,Arsenal
...,...,...,...,...,...,...,...
2200,1,1,2023-04-29,Wolverhampton Wanderers,Brighton,L,Wolves
2201,2,1,2023-05-06,Wolverhampton Wanderers,Aston Villa,W,Wolves
2202,1,1,2023-05-13,Wolverhampton Wanderers,Manchester Utd,L,Wolves
2203,0,1,2023-05-20,Wolverhampton Wanderers,Everton,D,Wolves


In [220]:
merged=combined.merge(combined, left_on=["date","new_team"],right_on=["date","opponent"])
merged.columns

Index(['actual_x', 'prediction_x', 'date', 'team_x', 'opponent_x', 'result_x',
       'new_team_x', 'actual_y', 'prediction_y', 'team_y', 'opponent_y',
       'result_y', 'new_team_y'],
      dtype='object')

In [225]:
merged[((merged['prediction_x']==1) & (merged['prediction_y']==2))]["actual_x"].value_counts()


1    209
0     90
2     80
Name: actual_x, dtype: int64

In [226]:
209/(209+80+90)

0.5514511873350924